# Extract data from a Sample Form which has been filled up with hand written text

## Prerequisites
1. To run the code, install the following packages. Please use the latest pre-release version `pip install azure-ai-formrecognizer==3.3.0`.


- > ! pip install azure-ai-formrecognizer==3.3.0

## Load all the API keys and parameters

In [1]:
import fr

MY_FORM_RECOGNIZER_ENDPOINT = 'https://tr-docai-form-recognizer.cognitiveservices.azure.com/'

formRecognizerCredential = fr.getFormRecognizerCredential()


Could not get API key from environment variable OPENAI_API_KEY. Trying Managed ID

Authenticated successfully with AAD token
AOAI setup succeeded
